What this code is doing:
1. It is taking the JSON file created in the workout generation algorithm and it is creating  a csv "Subfile" of that file that contains a feew specific columns of the JSON file. i have stored the JSON file with the name 'workout_plan.json'.
2. The "Subfile" csv file is named 'workout_plan.csv' and it contains the necessary columns of the workout plan that is; Day of the plan, Focus Area of that day, Main Workout Type of that day and the calories burnt on that day.
2.1. The focus area is used to determine the rest days primarily i.e the rest day will have different caloric intake and different macronutrient percentage.
2.2 The main workout Type i.e. Strength, HIIT and cardio etc. will be used to refine the macronutrients ratio i.e Strength workout will increase protein intake.
3. The workout type and calories are extracted from the workout_plans.csv file and are used to generate a weekly meal plan that is stored in the 'meal_plan_suggestions.csv' file. This file contains five columns; Day, Daily Calories and the three macronutrients in grams

In [37]:
# User input declaration and conversion functions for user inputs

# Global user variables
USER_AGE = None
USER_AGE_GROUP = None
USER_ACTIVITY_LEVEL = None
USER_REST_DAY = None
USER_PROGRAM_DURATION = None
USER_GOAL = None
USER_WEIGHT = None
USER_HEIGHT = None
USER_BMI = None

def map_age_to_group(age):
    if age < 40:
        return 'adult'
    elif 40 <= age < 60:
        return 'middle_aged'
    else:
        return 'older_adult'

def convert_activity_level(slider_value):
  #Assumptions for activity levels
  # 0 - 34 slider : low activity
  # 35 - 69 slider : moderate activity
  # 70 - 100 slider : high activity
    if slider_value < 35:
        return 'low'
    elif slider_value < 70:
        return 'moderate'
    else:
        return 'high'

def calculate_bmi(height_feet, weight_kg):
    # Convert height from feet to meters (1 foot = 0.3048 meters)
    height_meters = (height_feet * 0.3048)

    # Calculate BMI (weight in kg divided by height in meters squared)
    bmi = weight_kg / (height_meters ** 2)
    return round(bmi, 2)


def load_user_profile():
    global USER_AGE, USER_AGE_GROUP, USER_ACTIVITY_LEVEL, USER_REST_DAY, USER_PROGRAM_DURATION, USER_GOAL, USER_WEIGHT, USER_HEIGHT, USER_BMI

    # Simulated inputs from frontend (hardcoded for now)
    USER_AGE = 60
    activity_slider_value = 80  # integer slider value from 0 - 100
    USER_ACTIVITY_LEVEL = convert_activity_level(activity_slider_value)  # Converted to 'low', 'moderate', or 'high'
    USER_REST_DAY = 'Sunday'
    USER_PROGRAM_DURATION = 30
    USER_GOAL = 'stay_fit'
    USER_WEIGHT = 50 # in kgs
    USER_HEIGHT = 5.8 # in feet
    USER_BMI = calculate_bmi(USER_HEIGHT, USER_WEIGHT)

    # Map age to age group for downstream rules
    USER_AGE_GROUP = map_age_to_group(USER_AGE)

    print({
        "Age": USER_AGE,
        "Age Group": USER_AGE_GROUP,
        "Activity Level (Slider)": activity_slider_value,
        "Converted Activity Level": USER_ACTIVITY_LEVEL,
        "Preferred Rest Day": USER_REST_DAY,
        "Program Duration": USER_PROGRAM_DURATION,
        "Goal": USER_GOAL,
        "Weight": USER_WEIGHT,
        "Height": USER_HEIGHT,
        "BMI": USER_BMI
    })

# Test print
load_user_profile()


{'Age': 60, 'Age Group': 'older_adult', 'Activity Level (Slider)': 80, 'Converted Activity Level': 'high', 'Preferred Rest Day': 'Sunday', 'Program Duration': 30, 'Goal': 'stay_fit', 'Weight': 50, 'Height': 5.8, 'BMI': 16.0}


In [45]:
import json
import csv

#Read JSON file to generate the meal plan
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

#Generate CSV file with dominant workout type (highest calories burned)
def generate_csv(data, filename='workout_plan.csv'):
    csv_data = []
    for day_data in data:
        day = day_data["Day"]
        focus = day_data["Focus"]
        total_calories = day_data["Total Calories Burned"]

        # Get all workouts for the day with their types and calories
        workouts = day_data.get('Workouts', [])

        if not workouts:
            # No workouts, use default rest type prioritization
            dominant_type = "rest"
        else:
            # Find workout with maximum calories burned to finalize the workout type for meal refinements
            max_calories = -1
            dominant_type = ""
            for workout in workouts:
                if workout['Calories Burned'] > max_calories:
                    max_calories = workout['Calories Burned']
                    dominant_type = workout['Type']

        # Prepare the entry for each day
        csv_data.append({
            'Day': day,
            'Focus': focus,
            'Workout Type': dominant_type,  # Now shows only the dominant type
            'Total Calories Burned': total_calories
        })

    # Write to CSV
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['Day', 'Focus', 'Workout Type', 'Total Calories Burned'])
        writer.writeheader()
        writer.writerows(csv_data)

    print(f"CSV file '{filename}' has been generated successfully.")

def main():
    json_filename = 'workout_plan.json'
    data = read_json(json_filename)
    generate_csv(data)

if __name__ == "__main__":
    main()

CSV file 'workout_plan.csv' has been generated successfully.


In [50]:
#Generating meal plans and saving macronutrients in different columns
import csv

def read_workout_plan(csv_file):
    daily_workouts = {}
    with open(csv_file, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            day = row['Day']
            workout_type = row['Workout Type'].lower()
            calories_burned = float(row['Total Calories Burned'])
            daily_workouts[day] = (workout_type, calories_burned)
    return daily_workouts

#Calculate base calories using BMI and activity level
def adjust_calories(age, bmi, activity_level):
    if age < 30:
        bmr = 15.3 * bmi * 1.2 + 679
    elif age < 50:
        bmr = 11.6 * bmi * 1.2 + 879
    else:
        bmr = 13.5 * bmi * 1.2 + 487

    activity_multipliers = {
        "low": 1.2,
        "moderate": 1.55,
        "high": 1.9
    }
    return bmr * activity_multipliers.get(activity_level.lower(), 1.2)

#Adjust calories based on fitness goal
def adjust_meal_calories(base_calories, calories_burned, goal):
    if goal == "weight_loss":
        return max(1200, base_calories - 300 + calories_burned)
    elif goal == "muscle_gain":
        return base_calories + 250 + calories_burned
    elif goal == "maintenance":
        return base_calories + calories_burned
    return base_calories

#Get macro distribution based on workout type
def get_macronutrient_distribution(workout_type):
    macros_by_workout = {
        "strength": {"carbs": 40, "protein": 40, "fat": 20},
        "hiit": {"carbs": 50, "protein": 30, "fat": 20},
        "cardio": {"carbs": 55, "protein": 25, "fat": 20},
        "rest": {"carbs": 45, "protein": 30, "fat": 25}
    }
    return macros_by_workout.get(workout_type.lower(), macros_by_workout["rest"])

#Convert macro percentages to grams
def convert_macros_to_grams(total_calories, macro_distribution):
    calories_per_gram = {"carbs": 4, "protein": 4, "fat": 9}
    return {
        macro: round((percent / 100) * total_calories / calories_per_gram[macro], 1)
        for macro, percent in macro_distribution.items()
    }

#Generate meal plan for each day from workout_plan.csv
def generate_weekly_meal_plan(csv_file, age, bmi, activity_level, goal):
    daily_workouts = read_workout_plan(csv_file)
    weekly_plan = []

    for day, (workout_type, calories_burned) in daily_workouts.items():
        base_calories = adjust_calories(age, bmi, activity_level)
        daily_calories = adjust_meal_calories(base_calories, calories_burned, goal)
        macro_distribution = get_macronutrient_distribution(workout_type)
        macro_grams = convert_macros_to_grams(daily_calories, macro_distribution)

        weekly_plan.append({
            'Day': day,
            'Daily Calories': round(daily_calories, 2),
            'Carbs (g)': macro_grams['carbs'],
            'Protein (g)': macro_grams['protein'],
            'Fat (g)': macro_grams['fat']
        })

    return weekly_plan

#Save meal plan to CSV file with separate columns for each macronutrient
def save_meal_plan_to_csv(meal_plan, output_file='meal_plan_suggestions.csv'):
    with open(output_file, mode='w', newline='') as file:
        fieldnames = ['Day', 'Daily Calories', 'Carbs (g)', 'Protein (g)', 'Fat (g)']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(meal_plan)
    print(f"Meal plan saved to '{output_file}'")

# Example Usage
if __name__ == "__main__":
    CSV_FILE = "workout_plan.csv"  # Ensure this file exists
    OUTPUT_FILE = "meal_plan_suggestions.csv"

    # Generate weekly meal plan
    meal_plan = generate_weekly_meal_plan(
      CSV_FILE, USER_AGE, USER_BMI, USER_ACTIVITY_LEVEL, USER_GOAL
      )

    # Save to CSV
    save_meal_plan_to_csv(meal_plan, OUTPUT_FILE)

Meal plan saved to 'meal_plan_suggestions.csv'
